In [1]:
from dataReadFromDisk import load_df_from_csv,process_audio_and_beatmap_for_model_all_and_save,load_all_beatmaps_from_df,process_audio_and_beatmap_for_model_single

In [2]:
df = load_df_from_csv()

DataFrame loaded from: E:\\Notebooks\\osu\processed-metadata\processed-metadata.csv


In [3]:
# df = df[:50]
# df.iloc[7]
# df = df[df["audio"] == "0013ddfd8bd55fdccc0b253e313b7a60"]
# df

In [4]:
# testa,testb = process_audio_and_beatmap_for_model_single(df.iloc[0])
# testa.shape
# # for h in testb:
# #     print(h)

In [5]:
dataset,num_of_batches,processed_audio_list,processed_beatmap_list = process_audio_and_beatmap_for_model_all_and_save(df)

In [6]:
num_of_batches

49

In [7]:
processed_audio_list[0].shape

(23050, 142)

In [8]:
# from makeBeatmap import make_beatmap_from_output
# make_beatmap_from_output(df.iloc[0],processed_beatmap_list[0])

In [9]:
# for input_data, output_data in dataset.take(1):
#     print("First input:")
#     print(input_data.numpy()[0][0])
#     print("First output:")
#     print(output_data.numpy()[0][0])

In [10]:
# import numpy as np
# import tensorflow as tf

# model = tf.keras.Sequential([
#     tf.keras.layers.Input(shape=(None, 142)),  # Input layer for variable length sequences with 142 features
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256 , return_sequences=True)),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256 , return_sequences=True)),
#     # tf.keras.layers.Dense(64, activation='relu'),  # Optional dense layer
#     tf.keras.layers.Dense(35)  # Output layer, size 35 to match your output data dimension
# ])

# model.compile(optimizer='adam', loss=tf.keras.losses.Huber())

# # Training the model
# model.fit(dataset, epochs=10, steps_per_epoch=num_of_batches) # instead of 50 make it number of songs in final processed data

In [11]:
# import tensorflow as tf

# # Check if GPU is available
# gpus = tf.config.experimental.list_physical_devices()
# gpus

In [12]:
# print(tf.config.list_physical_devices('GPU'))

In [13]:
# tf.__version__

In [25]:
import tensorflow as tf

# # Ensure TensorFlow is set up to use the GPU
# physical_devices = tf.config.list_physical_devices('GPU')
# if len(physical_devices) > 0:
#     tf.config.experimental.set_memory_growth(physical_devices[0], True)
#     print("GPU is available and configured.")
# else:
#     print("GPU is not available.")

# Check for GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None, 142), ragged=True),  # Input layer for variable-length sequences with 142 features
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    # tf.keras.layers.Dense(64, activation='sigmoid'),  # Optional dense layer
    tf.keras.layers.Dense(35)  # Output layer, size 35 to match your output data dimension
])

# Compile the model
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=tf.keras.losses.Huber())

# Assuming `dataset` is already prepared and `num_of_batches` is defined
# Training the model

dataset = dataset.repeat()
with tf.device('/gpu:0'):
    model.fit(dataset, epochs=1, steps_per_epoch=num_of_batches)
# After running, you can monitor GPU usage using `nvidia-smi` from the command line


Num GPUs Available:  1
23/49 [=============>................] - ETA: 1:50 - loss: nan   

KeyboardInterrupt: 

In [17]:
from dotenv import load_dotenv
import os
#setting data path
load_dotenv()
project_path = os.getenv('PROJECT_PATH')
model.save(f'{project_path}/model_name.h5')

In [19]:
input_data = tf.ragged.constant([processed_audio_list[0]])  # Example input with sequence length 100 and 142 features

# Generate output using the loaded model
output = model.predict(input_data)

1/1 [==============================] - 5s 5s/step


In [23]:
for h in output[0]:
    print(h)

tf.Tensor(
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan], shape=(35,), dtype=float32)
tf.Tensor(
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan], shape=(35,), dtype=float32)
tf.Tensor(
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan], shape=(35,), dtype=float32)
tf.Tensor(
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan], shape=(35,), dtype=float32)
tf.Tensor(
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan], shape=(35,), dtype=float32)
tf.Tensor(
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 

KeyboardInterrupt: 

In [24]:
from makeBeatmap import make_beatmap_from_output
make_beatmap_from_output(df.iloc[0],output[0])

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment